## Hardware Details
[GCP](https://cloud.google.com/) VM: [n1-highmem-16](https://cloud.google.com/compute/docs/machine-types#n1_machine_types) (16 vCPUs, 104 GB memory)

In [1]:
system "lscpu"

"Architecture:          x86_64"
"CPU op-mode(s):        32-bit, 64-bit"
"Byte Order:            Little Endian"
"CPU(s):                16"
"On-line CPU(s) list:   0-15"
"Thread(s) per core:    2"
"Core(s) per socket:    8"
"Socket(s):             1"
"NUMA node(s):          1"
"Vendor ID:             GenuineIntel"
"CPU family:            6"
"Model:                 63"
"Model name:            Intel(R) Xeon(R) CPU @ 2.30GHz"
"Stepping:              0"
"CPU MHz:               2300.000"
"BogoMIPS:              4600.00"
"Hypervisor vendor:     KVM"
"Virtualization type:   full"
"L1d cache:             32K"
"L1i cache:             32K"
"L2 cache:              256K"
"L3 cache:              46080K"
..


In [2]:
system "cat /proc/meminfo | head -n1"

"MemTotal:       107091244 kB"


In [3]:
// nr of slaves
\s

16i


## Basic functions

In [4]:
createTable: {[tname; rowCount]
    .Q.gc[];
    tname set `bucket xasc ([] bucket: rowCount?`2; qty: rowCount?100h; risk: rowCount?10h; weight: rowCount?2.)
  }

In [5]:
executeParallel: {[tNames]
  :select bucket, NR, TOTAL_QTY, AVG_QTY: TOTAL_QTY % NR, 
    TOTAL_RISK, AVG_RISK: TOTAL_RISK % NR, W_AVG_QTY: W_SUM_QTY % TOTAL_WEIGHT, W_AVG_RISK: W_SUM_RISK % TOTAL_WEIGHT from 
    sum ?[; (); enlist[`bucket]!enlist `bucket; `NR`TOTAL_QTY`TOTAL_RISK`W_SUM_QTY`W_SUM_RISK`TOTAL_WEIGHT!((count; 1b); (sum; `qty); (sum; `risk); (wsum; `weight; `qty); (wsum; `weight; `risk); (sum; `weight))] peach tNames;
  }

In [6]:
N: abs system "s";
if[not 0 < N; '"Please start the q process with slave processes, e.g. 'q -s 16'"]
tableNamePrefix: "t";
tNames: `$tableNamePrefix ,' string til N

## 10k

In [7]:
rowNumber: 10 * 1000;
createTable[; rowNumber div N] each tNames;
\t:100 executeParallel tNames

276


## 100k

In [8]:
rowNumber: 100 * 1000;
createTable[; rowNumber div N] each tNames;
\t:100 executeParallel tNames

313


## 1M

In [9]:
rowNumber: 1000 * 1000;
createTable[; rowNumber div N] each tNames;
\t:100 executeParallel tNames

990


## 10M

In [10]:
rowNumber: 10 * 1000 * 1000;
createTable[; rowNumber div N] each tNames;
\t:100 executeParallel tNames

7838


## 100M
We execute the tests ten times only!

In [11]:
rowNumber: 100 * 1000 * 1000;
createTable[; rowNumber div N] each tNames;

In [12]:
\t:10 executeParallel tNames

8435


## 1B
We execute the tests ten times only!

In [13]:
rowNumber: 1000 * 1000 * 1000;
createTable[; rowNumber div N] each tNames

`t0`t1`t2`t3`t4`t5`t6`t7`t8`t9`t10`t11`t12`t13`t14`t15


In [14]:
\t:10 executeParallel tNames

67834
